In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/CutDebt/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(205, 2)


,label,split_text
0,0,可以
1,1,我 现在 实在 没 那么 多 钱
2,2,今天 天气 不错
3,0,行
4,0,好
5,0,没 问题 那 我 在 哪 天 之前 需要 还清 呢
6,0,这样 我 就 比较 可以 接受
7,0,嗯 嗯 这个 金额 可以 的
8,0,好 的 这个 金额 我 还 能 接受
9,0,这个 我 就 可以 还 了


In [6]:
data.split_text.iloc[89]

'这个 数额 可以'

In [7]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<205x751 sparse matrix of type '<class 'numpy.float64'>'
	with 1866 stored elements in Compressed Sparse Row format>

# SVC

In [8]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

1.0


In [9]:
print(clf.predict(phrase))

[0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 2 2 2 2 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2]


# Logistic

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

1.0


In [13]:
print(log_r.predict(phrase))

[0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 2 2 2 2 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2]


# LightGBM

In [14]:
import lightgbm as lgb

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [16]:
onelabels

array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [18]:
lables

array([[1, 1, 1]])

In [19]:
data.label.values

array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [20]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.302439
[10]	valid_0's multi_error: 0.287805
[15]	valid_0's multi_error: 0.287805
[20]	valid_0's multi_error: 0.287805
[25]	valid_0's multi_error: 0.263415
[30]	valid_0's multi_error: 0.263415
[35]	valid_0's multi_error: 0.263415
[40]	valid_0's multi_error: 0.263415
[45]	valid_0's multi_error: 0.263415
[50]	valid_0's multi_error: 0.263415
[55]	valid_0's multi_error: 0.263415
[60]	valid_0's multi_error: 0.263415
[65]	valid_0's multi_error: 0.263415
[70]	valid_0's multi_error: 0.263415
[75]	valid_0's multi_error: 0.263415
[80]	valid_0's multi_error: 0.263415
[85]	valid_0's multi_error: 0.263415
[90]	valid_0's multi_error: 0.263415
[95]	valid_0's multi_error: 0.263415
[100]	valid_0's multi_error: 0.263415
[105]	valid_0's multi_error: 0.263415
[110]	valid_0's multi_error: 0.263415
[115]	valid_0's multi_error: 0.263415
[120]	valid_0's multi_error: 0.263415
[125]	valid_0's multi_error: 0.263415
[130]	v

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [21]:
print(lgbm_model.predict(phrase))

[[ 0.57605964  0.47917986  0.0329834 ]
 [ 0.26421639  0.85889439  0.02948281]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.50676564  0.2834531   0.25838868]
 [ 0.76541817  0.21811974  0.02645144]
 [ 0.95008337  0.10054752  0.02061336]
 [ 0.9677108   0.0075527   0.07155331]
 [ 0.93266309  0.02580571  0.02242755]
 [ 0.97926791  0.00870277  0.0537524 ]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.97926791  0.00870277  0.0537524 ]
 [ 0.95831289  0.25232372  0.02512626]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.00697782  0.98567562  0.02084244]
 [ 0.12941547  0.88427192  0.03958269]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.38489577  0.31653818  0.25838868]
 [ 0.33609976  0.74798382  0.0329834 ]
 [ 0.36121689  0.74798382  0.0329834 ]
 [ 0.25357766  0.74798382  0.0329834 ]
 [ 0.00727371  0.98955091

# Save

In [23]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/CutDebt/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/CutDebt/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/CutDebt/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/CutDebt/Lgbm.pickle", "wb"))

In [32]:
test = phrase_vectorizer.transform(['真的 没 那么多'])
# test = phrase_vectorizer.transform(['我 在 洗澡'])


In [33]:
clf.predict_proba(test) # linear svc

array([[ 0.26294225,  0.62624623,  0.11081152]])

In [34]:
log_r.predict_proba(test) # logistic

array([[ 0.34298121,  0.42074601,  0.23627277]])

In [35]:
lgbm_model.predict(test) # light gbm

array([[ 0.38489577,  0.31653818,  0.25838868]])

# Labeling

In [36]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 2
prob= 0.626246225841
